In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gc
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df = pd.read_parquet('/kaggle/input/visual-taxonomy/category_attributes.parquet')
print(df.head())

In [ ]:
train_path = "/kaggle/input/visual-taxonomy/train.csv"
train_df = pd.read_csv(train_path)

In [ ]:
temp = train_df.groupby('Category').apply(len)

In [ ]:
print(temp)

In [ ]:
l = df['Attribute_list'][3]

In [ ]:
for item in l :
    print(item)

In [ ]:
df_women = train_df.loc[train_df['Category'] == 'Women Tshirts']
df_women.reset_index(drop=True, inplace=True)
# for i in range(1,9) :
#     temp = df_women.groupby(f'attr_{i}').apply(len)
#     print(temp) 
#     print('--------------------------------------------')
#     print('\n')

In [ ]:
from keras.utils import normalize, to_categorical
pd.options.mode.chained_assignment = None

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from tqdm import tqdm
from keras.preprocessing import image

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:

df_women['attr_5'] = df_women["attr_5"].astype('category').cat.codes


y_print_or_pattern_type = to_categorical(np.array(df_women['attr_5']))


In [ ]:
image_directory = "/kaggle/input/visual-taxonomy/train_images/"
SIZE=200
X_dataset = []  
for i in tqdm(range(df_women.shape[0])):
    prep = str(df_women['id'][i])
    prep = '0'*(6-len(prep)) + prep
    img = image.load_img(image_directory +prep+'.jpg', target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)
X = np.array(X_dataset)

In [ ]:

NUM_CLASSES_PP = len(df_women['attr_5'].unique())-1


In [ ]:
IMAGE_SIZE = (200, 200, 3)


# Load the base model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE)

# Freeze the base model
base_model.trainable = True

In [ ]:
print(len(base_model.layers))

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 130

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[0:]:
  layer.trainable = False

In [ ]:
import keras 

In [ ]:

# Split into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_print_or_pattern_type,
    test_size=0.2,
    random_state=42
)

In [ ]:
del X
del train_df 
gc.collect()

In [ ]:
NUM_CLASSES_PP = len(df_women['attr_5'].unique())-1

In [ ]:
# Split the combined labels back into their respective arrays
train_color_labels = y_train[:, :NUM_CLASSES_COLOR]
train_fs_labels = y_train[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_FS]
train_length_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH]
train_pattern_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_PATTERN]
train_pp_labels = y_train[:,  -NUM_CLASSES_SURFACE-NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP:-NUM_CLASSES_SURFACE-NUM_CLASSES_SS-NUM_CLASSES_SL]
train_sl_labels = y_train[:, -NUM_CLASSES_SURFACE-NUM_CLASSES_SS-NUM_CLASSES_SL:-NUM_CLASSES_SURFACE-NUM_CLASSES_SS]
train_ss_labels = y_train[:, -NUM_CLASSES_SURFACE-NUM_CLASSES_SS:-NUM_CLASSES_SURFACE]
train_surface_labels = y_train[:, -NUM_CLASSES_SURFACE:]

val_color_labels = y_val[:, :NUM_CLASSES_COLOR]
val_fs_labels = y_val[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_FS]
val_length_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH]
val_pattern_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FS + NUM_CLASSES_LENGTH + NUM_CLASSES_PATTERN]
val_pp_labels = y_val[:,  -NUM_CLASSES_SURFACE-NUM_CLASSES_SS-NUM_CLASSES_SL-NUM_CLASSES_PP:-NUM_CLASSES_SURFACE-NUM_CLASSES_SS-NUM_CLASSES_SL]
val_sl_labels = y_val[:, -NUM_CLASSES_SURFACE-NUM_CLASSES_SS-NUM_CLASSES_SL:-NUM_CLASSES_SURFACE-NUM_CLASSES_SS]
val_ss_labels = y_val[:, -NUM_CLASSES_SURFACE-NUM_CLASSES_SS:-NUM_CLASSES_SURFACE]
val_surface_labels = y_val[:, -NUM_CLASSES_SURFACE:]


In [ ]:
# resnet_model = Sequential()

# pretrained_model= tf.keras.applications.ResNet50(include_top=False,
#                    input_shape=(180,180,3),
#                    pooling='avg',classes=5,
#                    weights='imagenet')
# for layer in pretrained_model.layers:
#         layer.trainable=False

# resnet_model.add(pretrained_model)
# resnet_model.add(Flatten())
# resnet_model.add(Dense(512, activation='relu'))
# resnet_model.add(Dense(5, activation='softmax'))



# Create a model with five output heads for each attribute
inputs = layers.Input(shape=IMAGE_SIZE)
x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
# Define output layers for each attribute
output_pp = layers.Dense(NUM_CLASSES_PP, activation='softmax', name='pp')(x)

# Define the model
model = models.Model(inputs=inputs, outputs=[output_pp])

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5*1e-4),
    loss={
        'pp': 'categorical_crossentropy',
    },
    metrics=['accuracy']
)

# Summary of the model
model.summary()

In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

train_datagen = ImageDataGenerator(rotation_range=45,
    width_shift_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [ ]:
class MultiLabelDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size, datagen):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.datagen = datagen
        self.indices = np.arange(len(images))

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = self.images[batch_indices]
        batch_labels = {key: label[batch_indices] for key, label in self.labels.items()}
        
        # Apply data augmentation
        batch_images = np.array([self.datagen.random_transform(img) for img in batch_images])

        return batch_images, batch_labels

In [ ]:

train_labels = {
    'pp': y_train
}
batch_size = 32
train_generator = MultiLabelDataGenerator(X_train, train_labels, batch_size, train_datagen)

In [ ]:
checkpoint_filepath = '/kaggle/working/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_pp_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam

# Custom callback to progressively unfreeze layers
class GradualUnfreeze(Callback):
    def __init__(self,total_epochs, layers_to_unfreeze_per_epoch):
        super().__init__()
        self.total_epochs = total_epochs
        self.layers_to_unfreeze_per_epoch = layers_to_unfreeze_per_epoch

    def on_epoch_begin(self, epoch, logs=None):
        # Calculate number of layers to unfreeze based on epoch
        num_layers_to_unfreeze = min(25,(epoch) * self.layers_to_unfreeze_per_epoch)
        if num_layers_to_unfreeze :
            for layer in self.model.layers[-num_layers_to_unfreeze:]:
                layer.trainable = True
        lr = 5e-4 * (max(0.02, 1-(epoch/50)))
        self.model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
        self.model.make_train_function()
        print(f"Epoch {epoch+1}: Unfroze top {num_layers_to_unfreeze} layers. Learning Rate is {lr}")

In [ ]:
history = model.fit(
train_generator,  # Image data
validation_data=(X_val, {
'pp': y_val, 
}),
epochs=80,
callbacks=[model_checkpoint_callback, GradualUnfreeze(80,1)])

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the output from model.fit()
# Example: history = model.fit(...)

# Plotting the accuracy
plt.figure(figsize=(12, 6))

# Plot training accuracy

plt.plot(history.history['pp_accuracy'], label='Pattern Train Accuracy', color='green')

# Plot validation accuracy

plt.plot(history.history['val_pp_accuracy'], label='Pattern Validation Accuracy', linestyle='dashed', color='green')

# Adding labels and title
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
color
fit_shape
length
pattern
print_or_pattern_type
sleeve_length
sleeve_styling
surface_stylin

In [ ]:
import os 

In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'/kaggle/working/checkpoint.model.keras')

In [ ]:
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam

# Custom callback to progressively unfreeze layers
class GradualUnfreeze(Callback):
    def __init__(self, model, total_epochs, layers_to_unfreeze_per_epoch):
        super().__init__()
        self.model = model
        self.total_epochs = total_epochs
        self.layers_to_unfreeze_per_epoch = layers_to_unfreeze_per_epoch

    def on_epoch_begin(self, epoch, logs=None):
        # Calculate number of layers to unfreeze based on epoch
        num_layers_to_unfreeze = (epoch + 1) * self.layers_to_unfreeze_per_epoch
        for layer in self.model.layers[-num_layers_to_unfreeze:]:
            layer.trainable = True
        self.model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
        print(f"Epoch {epoch+1}: Unfroze top {num_layers_to_unfreeze} layers.")

# Load or define your model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(200, 200, 3))
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile model with frozen base layers
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Define number of epochs and layers to unfreeze per epoch
total_epochs = 20
layers_to_unfreeze_per_epoch = 2  # Adjust based on model depth and desired unfreeze speed

# Start training with the callback
history = model.fit(
    train_data,
    epochs=total_epochs,
    validation_data=validation_data,
    callbacks=[GradualUnfreeze(model, total_epochs, layers_to_unfreeze_per_epoch)]
)


In [ ]:
# resnet_model = Sequential()

# pretrained_model= tf.keras.applications.ResNet50(include_top=False,
#                    input_shape=(180,180,3),
#                    pooling='avg',classes=5,
#                    weights='imagenet')
# for layer in pretrained_model.layers:
#         layer.trainable=False

# resnet_model.add(pretrained_model)
# resnet_model.add(Flatten())
# resnet_model.add(Dense(512, activation='relu'))
# resnet_model.add(Dense(5, activation='softmax'))

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np

# class GradualUnfreeze(Callback):
#     def __init__(self,total_epochs, layers_to_unfreeze_per_epoch):
#         super().__init__()
#         self.total_epochs = total_epochs
#         self.layers_to_unfreeze_per_epoch = layers_to_unfreeze_per_epoch

#     def on_epoch_begin(self, epoch, logs=None):
#         # Calculate number of layers to unfreeze based on epoch
#         num_layers_to_unfreeze = min(25,(epoch + 1) * self.layers_to_unfreeze_per_epoch)
#         for layer in self.model.layers[-num_layers_to_unfreeze:]:
#             layer.trainable = True
#         lr = 5e-4 * (max(0.02, 1-(epoch/50)))
#         self.model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
#         print(f"Epoch {epoch+1}: Unfroze top {num_layers_to_unfreeze} layers. Learning Rate is {lr}")






class GradualUnfreezeCallback(Callback):
    """
    Callback to gradually unfreeze ResNet layers during training.
    Unfreezes layers from top to bottom (deepest to shallowest).
    """
    def __init__(self, base_model, initial_trainable_layers=0, layers_to_unfreeze_per_epoch=10):
        super().__init__()
        self.base_model = base_model
        self.layers_to_unfreeze_per_epoch = layers_to_unfreeze_per_epoch
        self.initial_trainable_layers = initial_trainable_layers
        self.learning_rates = {
            0: 1e-3,    # Initial learning rate for just the top layers
            10: 5e-4,   # Reduced learning rate when unfreezing begins
            30: 1e-4,   # Further reduced for fine-tuning deeper layers
            50: 5e-5    # Final fine-tuning
        }

    def on_epoch_begin(self, epoch, logs=None):
        # Calculate how many layers should be trainable
        trainable_layers = min(
            self.initial_trainable_layers + (epoch * self.layers_to_unfreeze_per_epoch),
            len(self.base_model.layers)
        )
        
        # Freeze/unfreeze layers accordingly
        for i, layer in enumerate(self.base_model.layers):
            layer.trainable = i >= (len(self.base_model.layers) - trainable_layers)
        
        # Adjust learning rate based on epoch
        for epoch_threshold, lr in self.learning_rates.items():
            if epoch >= epoch_threshold:
                current_lr = lr
                
        # Recompile the model with the new trainable layers and learning rate
        self.model.compile(
            optimizer=Adam(learning_rate=current_lr),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Print status
        trainable_params = np.sum([tf.keras.backend.count_params(w) for w in self.model.trainable_weights])
        print(f"\nEpoch {epoch + 1}:")
        print(f"Trainable layers: {trainable_layers}")
        print(f"Trainable parameters: {trainable_params:,}")
        print(f"Learning rate: {current_lr}")

def create_fine_tuning_model(num_classes, input_shape=(224, 224, 3)):
    """
    Creates a ResNet50 model for fine-tuning with custom top layers.
    """
    # Load pre-trained ResNet without top layers
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Add custom top layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=outputs)
    
    # Initially freeze all ResNet layers
    for layer in base_model.layers:
        layer.trainable = False
        
    return model, base_model

def train_with_gradual_unfreeze(
    train_data,
    validation_data,
    num_classes,
    input_shape=(224, 224, 3),
    epochs=100,
    batch_size=32,
    initial_trainable_layers=0,
    layers_to_unfreeze_per_epoch=10
):
    """
    Trains the model with gradual unfreezing of layers.
    """
    # Create model
    model, base_model = create_fine_tuning_model(num_classes, input_shape)
    
    # Create callbacks
    callbacks = [
        GradualUnfreezeCallback(
            base_model,
            initial_trainable_layers=initial_trainable_layers,
            layers_to_unfreeze_per_epoch=layers_to_unfreeze_per_epoch
        ),
        ModelCheckpoint(
            'best_model.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        EarlyStopping(
            monitor='val_accuracy',
            mode='max',
            patience=10,
            restore_best_weights=True,
            verbose=1
        )
    ]
    
    # Initial compilation
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train the model
    history = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, history


train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size = 32)

# Example usage:
"""
# Assuming you have your data prepared as tf.data.Dataset objects:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))

# Train the model
model, history = train_with_gradual_unfreeze(
    train_data=train_data,
    validation_data=validation_data,
    num_classes=10,
    epochs=100,
    batch_size=32,
    initial_trainable_layers=0,
    layers_to_unfreeze_per_epoch=10
)